In [2]:
import pandas as pd
import zipfile
import geopandas as gpd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def pluto(mappluto):
    z = zipfile.ZipFile(mappluto)
    z.extractall('../data_initial/')
    
    boroughs = ['Staten_Island', 'Queens', 'Manhattan', 'Bronx', 'Brooklyn']
    abreevs = ['SI', 'QN', 'MN', 'BX', 'BK']

    pluto = []
    i = 0
    for elem in boroughs:
        temp = gpd.read_file('../data_initial/'+elem+'/'+abreevs[i]+'MapPLUTO.shp')
        temp['BBL'] = temp['BBL'].astype(str)
        #temp = temp.to_crs({'init':'epsg:4326'})
        pluto.append(temp)
        i+=1  

    global pluto_updated
    pluto_updated = pd.concat(pluto)

    return pluto_updated.head()

In [3]:
pluto('../data_initial/mappluto_16v2.zip')

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,None,437 PURDY AVENUE,R3-1,None,7,15120.0,23400.0,5007470028.0,1488,...,0,1960,None,10314,R3-1,None,None,None,20d,"POLYGON ((944681.4200000018 161477.381400004, ..."
1,0.0,None,CUNARD AVENUE,R3A/HS,None,4,21518.0,21518.0,5006230252.0,0,...,0,0,None,10304,R3A,None,None,None,21d,"POLYGON ((959872.0577999949 162992.5866000056,..."
2,0.0,None,121 MOUNTAINVIEW AVENUE,R3-1,None,7,13561.0,22602.0,5007130017.0,1316,...,0,1920,None,10314,R3-1,None,None,None,21b,"POLYGON ((949278.1987999976 161346.9971999973,..."
3,0.0,None,26 FANNING STREET,R3X,None,7,9720.0,28740.0,5007140091.0,2188,...,0,1975,None,10314,R3X,None,None,None,21b,"POLYGON ((950061.0491999984 160815.3023999929,..."
4,0.0,None,335 SIMONSON AVENUE,R3A,None,7,7617.0,22118.0,5011840039.0,2392,...,0,1965,None,10303,R3A,None,None,None,20c,"POLYGON ((941981.4906000048 168120.2173999995,..."


In [161]:
pluto_updated.to_csv('../data_processed/pluto_16')

### there are no 'ownernames' with 'nyc housing authority' or 'nyc housing' so using ownertype instead

In [3]:
pluto_updated = pd.read_csv('../data_processed/pluto_16')

//anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (27,37,40,46,53,64,65,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
pluto_updated = pluto_updated.drop('Unnamed: 0',axis=1)

In [6]:
pluto_updated.head(2)

,APPBBL,APPDate,Address,AllZoning1,AllZoning2,AreaSource,AssessLand,AssessTot,BBL,BldgArea,...,YearAlter2,YearBuilt,ZMCode,ZipCode,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,ZoneMap,geometry
0,0.0,NaN,437 PURDY AVENUE,R3-1,NaN,7,15120.0,23400.0,5.007470e+09,1488,...,0,1960,NaN,10314,R3-1,NaN,NaN,NaN,20d,"POLYGON ((944681.4200000018 161477.381400004, ..."
1,0.0,NaN,CUNARD AVENUE,R3A/HS,NaN,4,21518.0,21518.0,5.006230e+09,0,...,0,0,NaN,10304,R3A,NaN,NaN,NaN,21d,"POLYGON ((959872.0577999949 162992.5866000056,..."


In [7]:
t1 = pluto_updated[pluto_updated['UnitsRes'] != 0]
t1 = t1[(t1['OwnerType'] != 'O') & (t1['OwnerType'] != 'C')]

In [8]:
len(t1)

758026

In [9]:
t2 = pluto_updated[pluto_updated['ResArea'] != 0]
t2 = t2[(t2['OwnerType'] != 'O') & (t2['OwnerType'] != 'C')]

In [10]:
len(t2)

756934

### selecting to use 'unitsres' as the value for not equalling zero because there are more rows included

In [11]:
t1.columns

Index([u'APPBBL', u'APPDate', u'Address', u'AllZoning1', u'AllZoning2',
       u'AreaSource', u'AssessLand', u'AssessTot', u'BBL', u'BldgArea',
       u'BldgClass', u'BldgDepth', u'BldgFront', u'Block', u'BoroCode',
       u'Borough', u'BsmtCode', u'BuiltCode', u'BuiltFAR', u'CB2010', u'CD',
       u'CT2010', u'ComArea', u'CommFAR', u'CondoNo', u'Council', u'EDesigNum',
       u'Easements', u'ExemptLand', u'ExemptTot', u'Ext', u'FacilFAR',
       u'FactryArea', u'FireComp', u'GarageArea', u'HealthArea', u'HistDist',
       u'IrrLotCode', u'LandUse', u'Landmark', u'Lot', u'LotArea', u'LotDepth',
       u'LotFront', u'LotType', u'LtdHeight', u'MAPPLUTO_F', u'NumBldgs',
       u'NumFloors', u'OfficeArea', u'OtherArea', u'Overlay1', u'Overlay2',
       u'OwnerName', u'OwnerType', u'PLUTOMapID', u'PolicePrct', u'ProxCode',
       u'ResArea', u'ResidFAR', u'RetailArea', u'SHAPE_Area', u'SHAPE_Leng',
       u'SPDist1', u'SPDist2', u'Sanborn', u'SchoolDist', u'SplitZone',
       u'StrgeArea', 

In [12]:
t1 = t1.drop(['APPBBL', 'APPDate', 'Address', 'AreaSource', 'BldgDepth', 'BldgFront', \
                                   'Block', 'BoroCode', 'Borough', 'CB2010', 'CD', 'CT2010', 'ComArea', \
                                   'CommFAR', 'CondoNo', 'Council', 'EDesigNum', 'FacilFAR', 'FactryArea', \
                                   'FireComp', 'GarageArea', 'HealthArea', 'HistDist', 'IrrLotCode', 'Landmark', \
                                   'Lot', 'LotArea', 'LotDepth', 'LotFront', 'LtdHeight', 'MAPPLUTO_F', \
             'OfficeArea', 'OtherArea', 'Overlay1', 'Overlay2', 'PLUTOMapID', 'PolicePrct', 'ResidFAR', \
             'RetailArea', 'Sanborn', 'SchoolDist', 'StrgeArea', 'TaxMap', 'Tract2010', 'YearAlter1', \
             'YearAlter2', 'ZoneDist1', 'ZipCode', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4', 'ZoneMap', \
             'XCoord', 'Version', 'YCoord', 'ZMCode', 'SPDist1', 'SPDist2', 'ExemptTot', 'BuiltCode', \
             'AllZoning1', 'AllZoning2', 'ExemptLand', 'geometry', 'Easements', 'SHAPE_Area', 'SHAPE_Leng', \
             'OwnerName', 'Ext', 'SplitZone'], axis=1)

In [13]:
t1.columns

Index([u'AssessLand', u'AssessTot', u'BBL', u'BldgArea', u'BldgClass',
       u'BsmtCode', u'BuiltFAR', u'LandUse', u'LotType', u'NumBldgs',
       u'NumFloors', u'OwnerType', u'ProxCode', u'ResArea', u'UnitsRes',
       u'UnitsTotal', u'YearBuilt'],
      dtype='object')

### explanation of variables in dataset
* assessland -  tentative assessed land value for Fiscal Year 2017
* assesstot - tentative assessed total value for Fiscal Year 2017
* bldgarea - total gross area in square feet
* bldgclass - code describing the major use of structures on the tax lot
* bsmtcode - code describing the basement type/grade - 0: no basement, 1: full basement above grade, 2: full basement below grade, 3: partial basement above grade, 4: partial basement below grade, 5: unknown
* landuse - code for the tax lot's land use category (01-09)
* lottype - code indicating the location of the tax lot to another tax lot and/or the water (1-9)
* numbldgs - number of buildings in the tax lot
* numfloors - the tallest building on the tax lot, the number of full and partial stories starting from the ground floor
* ownertype - M: mix city and private ownership, P: private ownership, X: mixed (excludes C, M, O, or P ownerships), blank: unknown
* proxcode - physical relationship of the building to neighboring buildings - 0: not available, 1: detatched, 2: semi-attached, 3: attached
* resarea - estimate of the exterior dimensions of the portion of the structure(s) allocated for residential use
* unitsres - sum of residential units in all buildings on the tax lot
* unitstotal - sum of residential and non-residential (offices, retail stores, etc.) units in all buildings on the tax lot
* yearbuilt - year structure was built

### merge pluto with complaint data

In [14]:
comp = pd.read_csv('../data_processed/311_heathotwater_2014_2016_BBL_WEEK_AVG.csv')
comp.head()

,BBL,com_count,BBL.1
0,1000047501,0.020408,1000047501
1,1000100032,0.006803,1000100032
2,1000150022,0.006803,1000150022
3,1000160015,0.006803,1000160015
4,1000160020,0.013605,1000160020


In [15]:
comp = comp.drop('BBL.1',axis=1)

In [16]:
print comp['BBL'].dtypes
print t1['BBL'].dtypes

int64
float64


In [17]:
import numpy as np

In [18]:
t1['BBL'] = pd.to_numeric(t1['BBL'])

In [19]:
bbl = np.array(t1['BBL'])
bbl = bbl.astype(int)

In [20]:
t1['BBL'] = bbl

In [21]:
print comp['BBL'].dtypes
print t1['BBL'].dtypes

int64
int64


In [97]:
features = pd.merge(t1, comp, on ='BBL')

In [98]:
print len(t1)
print len(comp)
print len(features)

758026
53310
51584


In [99]:
features.head()

,AssessLand,AssessTot,BBL,BldgArea,BldgClass,BsmtCode,BuiltFAR,LandUse,LotType,NumBldgs,NumFloors,OwnerType,ProxCode,ResArea,UnitsRes,UnitsTotal,YearBuilt,com_count
0,8441.0,22118.0,5010310021,4710,C0,2.0,0.70,2.0,5.0,1,2.75,NaN,1.0,4710,3,3,1920,0.013605
1,6480.0,24120.0,5012460037,2700,B2,2.0,0.92,1.0,5.0,1,2.00,NaN,1.0,2700,2,2,2003,0.034014
2,9007.0,30273.0,5033390088,1976,B1,2.0,0.48,1.0,5.0,1,2.00,NaN,1.0,1976,2,2,1955,0.054422
3,3980.0,11344.0,5000810024,1344,A1,2.0,0.54,1.0,5.0,1,2.00,NaN,1.0,1344,1,1,1901,0.074830
4,96750.0,324900.0,5000440001,14156,C9,5.0,0.66,2.0,3.0,1,2.00,P,0.0,14156,16,16,1984,0.006803


### select features with ensemble decision tree and random forest classifiers

In [100]:
features = features.drop('BBL',axis=1)

In [101]:
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [102]:
import random
random.seed(123)

In [103]:
le = preprocessing.LabelEncoder()

In [104]:
le.fit([elem for elem in features['BldgClass'].unique()])
features['bc_updated'] = le.transform([elem for elem in features['BldgClass']])
le.fit([elem for elem in features['OwnerType'].unique()])
features['ot_updated'] = le.transform([elem for elem in features['OwnerType']])

In [105]:
features = features.drop(['BldgClass', 'OwnerType'],axis=1)

In [106]:
features = features.apply(pd.to_numeric)
features = features.astype(int)

In [107]:
model = ExtraTreesClassifier()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.11071673  0.11950629  0.11934818  0.01950713  0.06470144  0.01336705
  0.02338104  0.01396726  0.03984421  0.00904866  0.11818101  0.10226136
  0.09252397  0.10804904  0.03502488  0.01057174]


In [108]:
etc = pd.DataFrame()
etc['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]].columns
etc['etc_value'] = model.feature_importances_

In [109]:
etc = etc.sort_values('etc_value',ascending=False)
etc

,feature_name,etc_value
1,AssessTot,0.119506
2,BldgArea,0.119348
10,ResArea,0.118181
0,AssessLand,0.110717
13,YearBuilt,0.108049
11,UnitsRes,0.102261
12,UnitsTotal,0.092524
4,BuiltFAR,0.064701
8,NumFloors,0.039844
14,bc_updated,0.035025


In [110]:
model = ExtraTreesRegressor()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.06675017  0.16882564  0.08875525  0.05414905  0.03796899  0.01022566
  0.01568029  0.00745825  0.03222633  0.02204506  0.10230521  0.09616619
  0.10465831  0.13995508  0.03439629  0.01843423]


In [111]:
etr = pd.DataFrame()
etr['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]].columns
etr['etr_value'] = model.feature_importances_

In [112]:
etr = etr.sort_values('etr_value',ascending=False)
etr

,feature_name,etr_value
1,AssessTot,0.168826
13,YearBuilt,0.139955
12,UnitsTotal,0.104658
10,ResArea,0.102305
11,UnitsRes,0.096166
2,BldgArea,0.088755
0,AssessLand,0.066750
3,BsmtCode,0.054149
4,BuiltFAR,0.037969
14,bc_updated,0.034396


In [113]:
model = RandomForestClassifier()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.13489076  0.16421958  0.14393727  0.02074796  0.03565899  0.01420324
  0.0198216   0.00887678  0.02231806  0.01676602  0.13241746  0.08531876
  0.07742929  0.07750673  0.02361959  0.0222679 ]


In [114]:
rfc = pd.DataFrame()
rfc['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]].columns
rfc['rfc_value'] = model.feature_importances_

In [115]:
rfc = rfc.sort_values('rfc_value',ascending=False)
rfc

,feature_name,rfc_value
1,AssessTot,0.164220
2,BldgArea,0.143937
0,AssessLand,0.134891
10,ResArea,0.132417
11,UnitsRes,0.085319
13,YearBuilt,0.077507
12,UnitsTotal,0.077429
4,BuiltFAR,0.035659
14,bc_updated,0.023620
8,NumFloors,0.022318


In [116]:
model = RandomForestRegressor()
model.fit(features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]], features.ix[:,-3])
# display the relative importance of each attribute
print(model.feature_importances_)

[ 0.15090698  0.27195039  0.10434389  0.01021782  0.01038726  0.00682277
  0.01596856  0.0021997   0.00992114  0.00603268  0.20742077  0.09367281
  0.03761044  0.03482665  0.03328303  0.00443512]


In [117]:
rfr = pd.DataFrame()
rfr['feature_name'] = features.ix[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16]].columns
rfr['rfr_value'] = model.feature_importances_

In [118]:
rfr = rfr.sort_values('rfr_value',ascending=False)
rfr

,feature_name,rfr_value
1,AssessTot,0.271950
10,ResArea,0.207421
0,AssessLand,0.150907
2,BldgArea,0.104344
11,UnitsRes,0.093673
12,UnitsTotal,0.037610
13,YearBuilt,0.034827
14,bc_updated,0.033283
6,LotType,0.015969
4,BuiltFAR,0.010387


In [119]:
merged = pd.merge(etc, rfc, on='feature_name')

In [120]:
merged = pd.merge(merged, rfr, on='feature_name')

In [121]:
merged = pd.merge(merged, etr, on ='feature_name')

In [122]:
merged

,feature_name,etc_value,rfc_value,rfr_value,etr_value
0,AssessTot,0.119506,0.164220,0.271950,0.168826
1,BldgArea,0.119348,0.143937,0.104344,0.088755
2,ResArea,0.118181,0.132417,0.207421,0.102305
3,AssessLand,0.110717,0.134891,0.150907,0.066750
4,YearBuilt,0.108049,0.077507,0.034827,0.139955
5,UnitsRes,0.102261,0.085319,0.093673,0.096166
6,UnitsTotal,0.092524,0.077429,0.037610,0.104658
7,BuiltFAR,0.064701,0.035659,0.010387,0.037969
8,NumFloors,0.039844,0.022318,0.009921,0.032226
9,bc_updated,0.035025,0.023620,0.033283,0.034396


In [125]:
features['com_count'].unique()

array([ 0,  1,  7,  3, 17,  2,  5, 30, 20,  9,  4,  6, 12, 10,  8])